In [1]:
import pycromanager as pyc
from pycromanager import Core,Studio
core = Core()
core.get_position()

-180

In [7]:
brushless = "KBD101_Thor_Rotation"
core.get_position(brushless)

276

In [9]:
import matplotlib.pyplot as plt
import tifffile as tf
import os
from fastprogress import progress_bar

In [3]:
def ppm_to_thor(angle):
    return (-2*angle + 276)

def thor_to_ppm(kinesis_pos):
    return (276 - kinesis_pos) / 2

In [ ]:
def set_angle(theta):
    theta = ppm_to_thor(theta)
    core.set_position(brushless,theta)
    core.wait_for_device(brushless)


In [8]:
from typing import Tuple,Dict,Any
import numpy as np
def snap_image() -> Tuple[np.ndarray, Dict[str, Any]]:
    core.snap_image()
    img = core.get_image()
    tags = {
        "Width": core.get_image_width(),
        "Height": core.get_image_height(),
        #"PixelType": core.get_pixel_type(),
        #"BytesPerPixel": core.get_bytes_per_pixel(),
        #"BitDepth": core.get_bitdepth(),
    }
    # Handle color cameras
    camera = core.get_property('Core','Camera')
    if camera in ["QCamera", "MicroPublisher6"]:
        if core.get_property(camera,'Color') == "ON":
            img = np.reshape(img, (tags["Height"], tags["Width"], 4))
            img = img[:, :, 0:3]  # Remove alpha
            img = np.flip(img, 2)  # Flip channels
        else:
            img = np.reshape(img, (tags["Height"], tags["Width"]))
    else:
        img = np.reshape(img, (tags["Height"], tags["Width"]))
    return img, tags


In [16]:
set_angle(0)

In [17]:
for k in progress_bar(range(0,360),total=360):
    set_angle(k)
    theta = core.get_position(brushless)
    img,tags = snap_image()
    filename = rf'D:\2025QPSC\purecalibration\20250701_calibration{k:03}.tif'
    tf.imwrite(filename,img.astype(np.uint16))    
    with open('log.txt','a') as fid:
        fid.write(str([k, theta, img.mean()]))


In [11]:
#img,tags = snap_image()

In [28]:
#plt.imshow(img,cmap='gray')
#plt.colorbar()

In [29]:
img.dtype

dtype('uint16')

In [30]:
img.shape

(2200, 2688)

# pure_calibration

'c:\\Users\\lociuser\\Codes\\smartpath\\smart-wsi-scanner\\src\\debeyering'

In [15]:
core.set_exposure(500)

In [16]:
for k in progress_bar([0,-5,5,90]):
    set_angle(k)
    if k == 90:
        core.set_exposure(10)
    else:
        core.set_exposure(500)
        
    theta = core.get_position(brushless)
    img,tags = snap_image()
    filename = rf'Micropublishertrial{k:.03f}_.tiff'
    tf.imwrite(filename,img.astype(np.uint16))    


In [ ]:
for k in progress_bar(range(0,360),total=360):
    set_angle(k)
    theta = core.get_position(brushless)
    img,tags = snap_image()
    filename = rf'D:\2025QPSC\purecalibration\Calibration_angle{k:03}.tif'
    tf.imwrite(filename,img.astype(np.uint16))    
    with open('log.txt','a') as fid:
        fid.write(str([k, theta, img.mean()]))

# 3 mins for full 360 deg calibration with 150 ms exposure